# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [52]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
docs = [
'feature one`feature two`feature three',
'feature one`feature two`feature four',
'feature one'
]

vocab = ['feature one', 'feature two', 'feature three', 'feature four']
vec = CountVectorizer(vocabulary=vocab)

# OR

vec = CountVectorizer(ngram_range=(2,2))

X = vec.fit_transform(docs)
vec.get_feature_names_out()

array(['feature four', 'feature one', 'feature three', 'feature two',
       'one feature', 'two feature'], dtype=object)

In [3]:
data = pd.read_csv('/content/labeled.csv')

In [4]:
data.head(10)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [5]:
pip install razdel

In [6]:
from razdel import tokenize
vectorizer_default = CountVectorizer()
vectorizer_razdel = CountVectorizer(tokenizer=lambda text: [token.text for token in tokenize(text)])

In [7]:
train, test = train_test_split(data, test_size=0.3, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [8]:
# СЧИТАЮ ДЛЯ vectorizer_default
X_def = vectorizer_default.fit_transform(train.comment)
X_test_def = vectorizer_default.transform(test.comment)

y_def = train.toxic.values
y_test_def = test.toxic.values

X_def.shape, X_test_def.shape

((10088, 54407), (4324, 54407))

In [9]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_def, y_def)
clf.classes_
preds = clf.predict(X_test_def)

In [10]:
print(classification_report(y_test_def, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87      2880
         1.0       0.71      0.84      0.77      1444

    accuracy                           0.83      4324
   macro avg       0.81      0.83      0.82      4324
weighted avg       0.84      0.83      0.83      4324



In [11]:
# СЧИТАЮ ДЛЯ vectorizer_razdel

X_raz = vectorizer_razdel.fit_transform(train.comment)
X_test_raz = vectorizer_razdel.transform(test.comment)

y_raz = train.toxic.values
y_test_raz = test.toxic.values

X_raz.shape, X_test_raz.shape

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


((10088, 55187), (4324, 55187))

In [ ]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_raz, y_raz)
clf.classes_
preds = clf.predict(X_test_raz)

In [13]:
print(classification_report(y_test_raz, preds, zero_division=0))
# Вывод: примерно одинаково

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87      2880
         1.0       0.71      0.83      0.77      1444

    accuracy                           0.83      4324
   macro avg       0.81      0.83      0.82      4324
weighted avg       0.84      0.83      0.83      4324



## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [14]:
# COUNT_VEC + LogReg

stop = list(stopwords.words('russian'))

count_vec = CountVectorizer(lowercase=True, encoding='UTF-8', min_df=1, tokenizer=lambda text: [token.text for token in tokenize(text)], stop_words=stop)

X = count_vec.fit_transform(train.comment)
X_test = count_vec.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [23]:
clf = LogisticRegression(C=0.5, class_weight='balanced', max_iter=100, penalty='l1', solver='saga')
clf.fit(X, y)
clf.classes_
preds = clf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [24]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.76      0.81      2880
         1.0       0.61      0.75      0.67      1444

    accuracy                           0.76      4324
   macro avg       0.73      0.75      0.74      4324
weighted avg       0.77      0.76      0.76      4324



In [25]:
percents_top10 = list(np.sort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
indices_top10  = list(np.argsort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
top10_sents = [count_vec.inverse_transform(X_test[i]) for i in indices_top10]
top10_sents

[[array(['!', '(', ',', '-', '.', '00', '2', '2019', '22', '23', '25', '5',
         '6', ':', '?', '?!', 'b', 'ch', 'd', 'hk', 'm', 'абу', 'аватарка',
         'аватарок', 'аниме', 'анон', 'анонов', 'ахуенно', 'бан', 'банит',
         'бля', 'блядь', 'бред', 'вебм', 'вебмки', 'видео', 'видимо', 'вне',
         'вообще', 'вопросы', 'вроде', 'второй', 'выпей', 'выписывают',
         'глаза', 'говоря', 'давайте', 'дал', 'данных', 'даунов', 'двач',
         'две', 'дебилов', 'деньги', 'должности', 'друг', 'другом',
         'ебанутые', 'единственное', 'ещё', 'желанию', 'жертвой', 'жизни',
         'жопу', 'забанили', 'застал', 'затем', 'злоупотребления', 'значит',
         'идти', 'изгнать', 'истекает', 'итоге', 'каких', 'какого', 'какое',
         'капчу', 'которые', 'который', 'кстати', 'логика', 'могут',
         'модератор', 'модератора', 'мочи', 'мочух', 'написав',
         'немедленно', 'нему', 'несправедливый', 'никаких', 'никто',
         'новом', 'обсуждение', 'обсуждения', 'обща

In [26]:
# TF-IDF_VEC + KNN

vectorizer = TfidfVectorizer(lowercase=True, encoding='UTF-8', min_df=1, tokenizer=lambda text: [token.text for token in tokenize(text)], stop_words=stop)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

X.shape, X_test.shape

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


((10088, 55036), (4324, 55036))

In [27]:
clf = KNeighborsClassifier(n_neighbors=10)

clf.fit(X, y)

preds = clf.predict(X_test)

In [28]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.79      0.96      0.86      2880
         1.0       0.85      0.48      0.61      1444

    accuracy                           0.80      4324
   macro avg       0.82      0.72      0.74      4324
weighted avg       0.81      0.80      0.78      4324



In [29]:
percents_top10 = list(np.sort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
indices_top10  = list(np.argsort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
top10_sents = [count_vec.inverse_transform(X_test[i]) for i in indices_top10]
top10_sents

[[array(['сука', '.'], dtype='<U67')],
 [array(['тредом', 'существу', 'политач', 'покинуть', 'пидор', 'должны',
         'ваша', '.', ','], dtype='<U67')],
 [array(['хохлов', 'тему', 'стрелки', 'кого', 'жидов', '?', '...', ','],
        dtype='<U67')],
 [array(['хохлов', 'хотели', 'т', 'сказал', 'реального', 'против',
         'посадить', 'оружия', 'лднр', 'е', 'выложил', 'вместо', 'бтр', '?',
         '.', ','], dtype='<U67')],
 [array(['явления', 'своими', 'просто', 'пидор', 'называть', 'именами',
         'давайте', 'вещи', '...', ',', '!'], dtype='<U67')],
 [array(['просто', 'дегенерат', '.'], dtype='<U67')],
 [array(['москали', 'лице', 'кожу', 'злые', 'ебать'], dtype='<U67')],
 [array(['похуй', 'всем', '.'], dtype='<U67')],
 [array(['хохол', 'москаль', '?', ',', '!'], dtype='<U67')],
 [array(['хохлов', 'раза'], dtype='<U67')]]

In [ ]:
# Вывод: Пердложения абсолютно разные, но все являются токсичными

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [87]:
# LogReg
count_vec = CountVectorizer(stop_words=stop)

X = count_vec.fit_transform(train.comment)
X_test = count_vec.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.5, class_weight='balanced', max_iter=10, penalty='l1', solver='saga')
clf.fit(X, y)
clf.classes_

preds = clf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [88]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.72      0.79      2880
         1.0       0.58      0.77      0.66      1444

    accuracy                           0.74      4324
   macro avg       0.72      0.75      0.73      4324
weighted avg       0.77      0.74      0.75      4324



In [91]:
top_index = []

for i in np.sort(clf.coef_[0])[::-1][:12]:
  top_index.append(list(clf.coef_[0]).index(i))
# оставим только слова (уберем "!" и "?")
count_vec.get_feature_names_out()[top_index][2:7]

array(['нахуй', 'хохлов', 'блядь', 'пиздец', 'хуй'], dtype=object)

In [84]:
# KNN
count_vec = TfidfVectorizer(stop_words=stop)

X = count_vec.fit_transform(train.comment)
X_test = count_vec.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [92]:
clf = RandomForestClassifier(n_estimators=300, max_depth=500)

clf.fit(X, y)

preds = clf.predict(X_test)
# если честно, то не знаю как еще улучшить f1....

In [93]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.77      0.98      0.86      2880
         1.0       0.89      0.41      0.56      1444

    accuracy                           0.79      4324
   macro avg       0.83      0.69      0.71      4324
weighted avg       0.81      0.79      0.76      4324



In [95]:
top_index = []

for i in np.sort(clf.feature_importances_)[::-1][:20]:
  top_index.append(list(clf.feature_importances_).index(i))
count_vec.get_feature_names_out()[top_index][:5]

array(['хохлы', 'хохлов', 'тебе', 'очень', 'нахуй'], dtype=object)